In [3]:
import pubchempy as pcp
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
import requests
import urllib.parse  # For URL encoding

In [5]:
def iupac_to_smiles(iupac_name: str):
    # URL encode the IUPAC name to handle special characters
    encoded_name = urllib.parse.quote(iupac_name)
    
    # PubChem PUG REST API URL
    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{encoded_name}/json'
    
    # Send the GET request to the API
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        # Check if the response contains the expected compound data
        if 'PC_Compounds' in data and len(data['PC_Compounds']) > 0:
            compound = data['PC_Compounds'][0]

            # Extract the SMILES string from the compound data
            if 'props' in compound:
                for prop in compound['props']:
                    if prop.get('urn', {}).get('label') == 'SMILES':
                        return prop['value']['sval']
            
            return f"SMILES not found for molecule '{iupac_name}'."
        else:
            return f"No compound data found for molecule '{iupac_name}'."
    else:
        return f"Error fetching data from PubChem (status code: {response.status_code})."

# Example usage
iupac_name = "propanol"
smiles = iupac_to_smiles(iupac_name)
print(f"SMILES for {iupac_name}: {smiles}")



SMILES for propanol: CCCO


In [59]:
url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/1234/JSON"

# Send the GET request to the API
response = requests.get(url)
print(response.status_code)
data = response.json()
import json
with open("test.json", "w") as f:
    json.dump(data, f, indent=4)

200


In [60]:
url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/ethanol/JSON"

# Send the GET request to the API
response = requests.get(url)
print(response.status_code)
data = response.json()
import json
with open("test2.json", "w") as f:
    json.dump(data, f, indent=4)

400


In [63]:
import requests

def predict_nmr_shifts(smiles, nucleus="1H"):
    """
    Predict NMR shifts using the NMRShiftDB API.

    Args:
        smiles (str): The SMILES string of the compound.
        nucleus (str): The nucleus to predict (e.g., "1H" or "13C").

    Returns:
        list: Predicted NMR shifts or an error message.
    """
    url = "https://nmrshiftdb.nmr.uni-koeln.de/rest/predictor/predict"
    params = {"structure": smiles, "type": nucleus}
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data.get("shifts", [])
    else:
        print(f"Error: {response.status_code}, Message: {response.text}")
        return None

# Example
smiles = "CCO"  # Ethanol
predicted_shifts = predict_nmr_shifts(smiles)
print(f"Predicted {smiles} NMR Shifts: {predicted_shifts}")


SSLError: HTTPSConnectionPool(host='nmrshiftdb.nmr.uni-koeln.de', port=443): Max retries exceeded with url: /rest/predictor/predict?structure=CCO&type=1H (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))

In [4]:
from rdkit import Chem

def iupac_to_smiles(iupac_name):
    """
    Convert an IUPAC name to a SMILES string using RDKit.
    
    Args:
        iupac_name (str): The IUPAC name of the molecule.
    
    Returns:
        str: The SMILES representation of the molecule, or an error message if conversion fails.
    """
    try:
        # Use RDKit to generate a molecule from the IUPAC name
        mol = Chem.MolFromName(iupac_name)
        if mol is None:
            return "Error: Could not parse the IUPAC name."
        
        # Convert the molecule to SMILES format
        smiles = Chem.MolToSmiles(mol)
        return smiles
    except Exception as e:
        return f"An error occurred: {e}"

# Example usage
iupac_name = "ethanol"  # Replace with any IUPAC name
smiles = iupac_to_smiles(iupac_name)
print(f"SMILES for {iupac_name}: {smiles}")

SMILES for ethanol: An error occurred: module 'rdkit.Chem' has no attribute 'MolFromName'
